In [1]:
import sys
sys.path.insert(0, '..')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from diaphanous import (
    REPORTS_PER_PLATFORM,
    combine_brands,
    ingest_reports_per_platform,
    show,
)

def logger(format: str, *args, **kwargs) -> None:
    print(format.format(*args, **kwargs))

data = ingest_reports_per_platform(REPORTS_PER_PLATFORM, logger=logger)

Skipping metadata
➖ Alphabet (no CSAM data)
✅ Amazon
➖ Apple (no CSAM data)
➖ Automattic (no CSAM data)
➖ Aylo (no CSAM data)
✅ Discord
✅ Facebook
✅ GitHub
✅ Google
✅ Instagram
✅ LinkedIn
➖ Meta (no CSAM data)
✅ Microsoft
➖ MindGeek (no transparency disclosures)
➖ Omegle (no CSAM data)
✅ Pinterest
✅ Pornhub
➖ Quora (no CSAM data)
✅ Reddit
✅ Snap
➖ Telegram (no transparency disclosures)
✅ TikTok
✅ TikTok (original schema)
➖ Tumblr (no CSAM data)
✅ Twitch
✅ Twitter
➖ WhatsApp (no CSAM data)
➖ Wikimedia (no transparency disclosures)
➖ Wordpress (no CSAM data)
➖ X (no CSAM data)
✅ YouTube
✅ NCMEC


In [2]:
ms = data.disclosures["Microsoft"]
ms["account_reversals"] = ms["accounts"] * ms["reinstated accounts"] / 100
ms = ms.drop(columns=["pieces", "automatically detected pieces", "reinstated accounts", "reports"])
ms = ms.groupby(ms.index.year).sum()
ms["percent_acounts_reversed"] = ms["account_reversals"] / ms["accounts"] * 100

fb = data.disclosures["Facebook"]
ig = data.disclosures["Instagram"]

META_REVERSALS = [
    "reversals (Child Nudity & Sexual Exploitation)",
    "reversals (Child Endangerment: Sexual Exploitation)",
    "reversals w/o appeal (Child Nudity & Sexual Exploitation)",
    "reversals w/o appeal (Child Endangerment: Sexual Exploitation)",
]

META_ACTIONED = [
    "pieces (Child Nudity & Sexual Exploitation)",
    "pieces (Child Endangerment: Sexual Exploitation)",
]

meta = pd.DataFrame(data={
    'actioned': fb[META_ACTIONED].sum(axis=1).groupby(fb.index.year).sum(),
    'reversed': fb[META_REVERSALS].sum(axis=1).groupby(fb.index.year).sum(),
})
meta["actioned"] += ig[META_ACTIONED].sum(axis=1).groupby(ig.index.year).sum()
meta["reversed"] += ig[META_REVERSALS].sum(axis=1).groupby(ig.index.year).sum()
meta["percent_pieces_reversed"] = meta["reversed"] / meta["actioned"] * 100

# Sigh, Pinterest has periods at quarter and half-year resolution. So Pandas
# doesn't create a PeriodIndex. So index.year doesn't work. Hence we first
# reduce the data to just the cells we need, explicitly create a PeriodIndex,
# group and aggregate, and then compute percentage

pt = data.disclosures["Pinterest"][["accounts", "account reversals"]]
pt = pt[~pt["accounts"].isna()]
pt.index = pd.PeriodIndex(pt.index)
pt = pt[pt.index.year >= 2022]
pt = pt.groupby(pt.index.year).sum()
pt["percent_accounts_reversed"] = pt["account reversals"] / pt["accounts"] * 100

# Let's assemble the concise results. Pinterest's numbers are terrifying:

show(
    """
    <h1>False Positives for CSE</h1>

    <p>We can approximate the false positive rate by taking the fraction of
    pieces or accounts that were actioned for CSE or CSAM but then restored
    again, whether by the platform on its own initiative or after the user
    launched an appeal. Unfortunately, only Meta, Microsoft, and Pinterest
    report the necessary statistics. The table below shows the results.</p>
    """
)

show(pd.DataFrame({
    "Meta (% Pieces)": meta["percent_pieces_reversed"],
    "Microsoft (% Accounts)": ms["percent_acounts_reversed"],
    "Pinterest (% Accounts)": pt["percent_accounts_reversed"],
}), caption="Percentage of Pieces/Accounts Restored")


,Meta (% Pieces),Microsoft (% Accounts),Pinterest (% Accounts)
period,,,
2018,⋯,⋯,⋯
2019,0.1153,⋯,⋯
2020,0.0738,0.00478,⋯
2021,0.3062,0.02777,⋯
2022,1.0941,0.96313,11.6
2023,3.1300,1.05285,12.8
2024,1.3875,⋯,⋯
